# GeoTIFF
GeoTIFF (Georeferenced Tagged Image File Format) is an public **raster** data file format```.tif```. It embeds spatial(georeferencing) information within a **TIFF** file. The potential additional information includes **map projection**, **coordinate systems**, **ellipsoids**, **datums** and everything else necessary to establish the exact spatial reference for the file. 

## 1. What is raster format?
There are two main type of image files: **raster** and **vector**. [cited here](https://vector-conversions.com/vectorizing/raster_vs_vector.html) 
* Raster images are created with pixel-based programs or captured with a camera or scanner (e.g. jpg, gif, png). A pixel is a single point or the smallest single element in a display device. If you zoom in to a raster image, you may start to see a lot of little tiny squares.
* Vector graphics are created with vector software and are commonly applied onto a physical product (e.g. used in CAD, engineering). Vector images are mathematical calculations from one point to another that form lines and shapes. If you zoom into a vector graphic, it will always look the same. 

## 2. From TIFF to GeoTIFF
[cited here](https://www.geospatialworld.net/article/geotiff-a-standard-image-file-format-for-gis-applications/).

TIFF(Tagged Image File Format) stores raster images(```.tif```). In each .tif file, there are raster data layer(s)(can have more than one raster image) with a set of TIFF tags. Tags can indicate some basic geometry info of the image, such as width, length, compression scheme used, colorspace, resolution, etc. 

GeoTIFF embeds extra geographic information for GIS (Geographic information system) usage, such as latitude, longitude, projected coordinate systems, model space, etc. It explains: 
* what area does this dataset cover?
* what spatial projection/coordinate reference system is used to store the data?
* what area on the ground does each pixel cover? what is its spatial resolution?
* how many layers are in the .tif file?

## 3. Some knowledge of Geographic
### [Geographic Coordinate System](https://en.wikipedia.org/wiki/Geographic_coordinate_system)
Geographic Coordinate System is a coordinate system with latitude, longitude and height. Earth is not a sphere, but an irregular shape approximating a biaxial ellipsoid. It is nearly spherical. Also the surface of earch is not flat. In order to be unambiguous about the vertical and surface when we are describing the earth, we need to choose a **reference ellipsoid** and a **datum**. 

### [Map Projection](http://kartoweb.itc.nl/geometrics/Map%20projections/body.htm)
The Eurth's surface is curved in a specific way. To represent some part of the planet to a flat two-dimensional paper map or on a computer screen requires a map projection. Mpping onto a 2D plane means transforming each point on the reference surface with geographic coordinates to a set of Cartesian coordinates(x, y) representing positions on the map plane. Different mapping equations have different parameters.

Different map projections can be described in terms of their: see [here](http://kartoweb.itc.nl/geometrics/Map%20projections/body.htm)
1. class (cylindrical, conical or azimuthal)
2. point of secancy (tangenent or secant
3. aspect (normal, transverse or oblique), and
4. distortion property (equal-area, equidistant or conformal).

Another descriptor of a map projection might be the name of the inventor (or first publisher) of the projection, such as Mercator, Lambert, Robinson, Cassini etc.. For example:
* Lambert conformal conic projection
* Lambert azimuthal equal-area projection
* Polar stereographic azimuthal projection

There are lots of different ways to describe a projection:
* [Well Known Text (WKT)](http://en.wikipedia.org/wiki/Well-known_text)
```
PROJCS["UTM Zone 12, Northern Hemisphere",GEOGCS["WGS_1984",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.2572235630016],TOWGS84[0,0,0,0,0,0,0]],PRIMEM[ "Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EP SG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["lati tude_of_origin",0],PARAMETER["central_meridian",- 111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting ",500000],PARAMETER["false_northing",0],UNIT["Meter",1],AUTHOR ITY["EPSG","32612"]]
```
* [PROJ.4](https://en.wikipedia.org/wiki/PROJ.4)
```
+proj=utm +zone=12 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
```
* [EPSG (European Petroleum Survey Group)](https://en.wikipedia.org/wiki/EPSG)
```
EPSG:32612
```
* [USGS (United States Geological Survey)](https://en.wikipedia.org/wiki/United_States_Geological_Survey)
```
(1, 12, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 12)
```
* [ESRI .prj file](https://vsp.pnnl.gov/help/vsample/ESRI_PRJ_File.htm)(Environmental Systems Research Institute)
* PCI software
```
('UTM 12 D000', 'METRE', (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))
```
* XML

## 4. Reading/Writing GeoTIFF (GDAL)
[GDAL](http://gdal.org) (Geospatial Data Abstraction Library) is an open source translator library for raster and vector geospatial data formats. The related OGR library (OGR simple features library) is part of the GDAL source tree, provides a similar ability for simple features vector graphics data.

### Install GDAL
*Directly using apt-get to install gdal-bin will get an old version of libgdal-dev (v. 1.11.3), which includes some development packages for different languages.*

To get the latest GDAL/OGR version, add the PPA to your sources, then install the gdal-bin package (this should automatically grab any necessary dependencies, including at least the relevant libgdal version). 

In [ ]:
! sudo add-apt-repository ppa:ubuntugis/ppa
! sudo apt-get update
! sudo apt-get install gdal-bin

#### Command line utility tools

```gdalinfo``` report information about a file

```gdal_traslate``` copy a raster file, with control of output format

**Some tips of the current project**

*I tried to use ```gdal_translate``` to achieve GeoTiff to NetCDF at the beginning. However, the auto translation can not satisfy the requirement of the desired NetCDF dataset for a couple reasons.*

1. The data variable need to be meaningfully named. Its attributes should be manually added to conform CF conventions.
2. An unlimited time dimension needs to be added. Moreover, the time dimension needs to be added to the data variable. 
3. To conform CF conventions, the latitude and longitude auxiliary coordinate variables should be supplied. It means that we need to unproject the projection coordinate system of GeoTiff. 
4. The auto translation includes some unnecessary TIFF TAGS, which were asked to be removed. 


### Python GDAL/OGR

Some useful examples:
* [Python GDAL/OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/)
* [Posts on osgeo.cn(Chinese)](http://www.osgeo.cn/leaf/7156g)
* [python gdal tutorial(Chinese)](http://www.osgeo.cn/python_gdal_utah_tutorial/ch05.html)

In [24]:
import sys
import numpy as np
## before v1.5
import gdal
## after v1.6
from osgeo import gdal

fn = '../UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif'
## open dataset
ds = gdal.Open(fn)

## Handle error
if ds is None:
    print ('Could not open ' + fn)
    sys.exit(1)

#### Get Metadata
Metadata includes: (*Not all of these TIFF TAGS are supposed to be shown in netCDF dataset)
* TIFFTAG_DOCUMENTNAME
* TIFFTAG_IMAGEDESCRIPTION
* TIFFTAG_SOFTWARE
* TIFFTAG_DATETIME
* TIFFTAG_ARTIST
* TIFFTAG_HOSTCOMPUTER
* TIFFTAG_COPYRIGHT
* TIFFTAG_XRESOLUTION
* TIFFTAG_YRESOLUTION
* TIFFTAG_RESOLUTIONUNIT
* TIFFTAG_MINSAMPLEVALUE (read only)
* TIFFTAG_MAXSAMPLEVALUE (read only)

In [2]:
ds.GetMetadata()

{'AREA_OR_POINT': 'Area',
 'TIFFTAG_DATETIME': '2018:02:13 13:18:46',
 'TIFFTAG_DOCUMENTNAME': 'C:\\Users\\h2kheyro\\Documents\\Data_Analysis\\Great Lakes\\LST\\MonthlyAve2/UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif',
 'TIFFTAG_IMAGEDESCRIPTION': 'IDL TIFF file',
 'TIFFTAG_RESOLUTIONUNIT': '2 (pixels/inch)',
 'TIFFTAG_SOFTWARE': 'IDL 7.1.2, ITT Visual Information Solutions',
 'TIFFTAG_XRESOLUTION': '100',
 'TIFFTAG_YRESOLUTION': '100'}

#### Get Raster Description

In [4]:
ds.GetDescription()

'../UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif'

#### Get the count of raster images

In [36]:
band_cnt = ds.RasterCount
band_cnt

1

#### Get size of the image (number of pixels on X axis and Y axis)

In [8]:
img_width = ds.RasterXSize
img_height = ds.RasterYSize
img_width, img_height

(1299, 1956)

#### Get Georeference Info (GeoTransform)
```GeoTransform()``` is a list, storing the pixel location of a geospatial coordinate. 
* ```GeoTransform()[0]```  top left x 
* ```GeoTransform()[1]```  west-east direction pixel resolution
* ```GeoTransform()[2]```  rotation, **0 if image is "north up"** 
* ```GeoTransform()[3]```  top left y 
* ```GeoTransform()[4]```  rotation, 0 if image is "north up" /*如果北边朝上，地图的旋转角度*/
* ```GeoTransform()[5]```  north-south direction pixel resolution

**NOTE**: the north-sourth direction resolution is a negative value.

In [22]:
x_topleft, x_res, dx_dy, y_topleft, dy_dx, y_res= ds.GetGeoTransform()
x_topleft, x_res, dx_dy, y_topleft, dy_dx, y_res

(-5386000.000000001, 1000.0, 0.0, 470999.99999999953, 0.0, -1000.0)

#### Get x and y values
To get the pixel offsets of a particular pixel(x, y), which is the offset from the top left pixel (suppose we know x and y)

In [ ]:
x_offset = int((x - x_topleft) / x_res)
y_offset = int((y - y_topleft) / y_res)

To get the x array and y array

In [59]:
xcoordinates = np.arange(start=x_topleft, stop=x_topleft+x_res*img_width, step=x_res)
ycoordinates = np.arange(start=y_topleft, stop=y_topleft+y_res*img_height, step=y_res)
xcoordinates, ycoordinates

(array([-5386000., -5385000., -5384000., ..., -4090000., -4089000.,
        -4088000.]),
 array([  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.]))

#### Get Raster Band(s)
```GetRasterBand(band_num)``` The ```band_num``` starts from ```1```. 

In [35]:
band = ds.GetRasterBand(1)
data = band.ReadAsArray()
len(data), len(data[0])

(1956, 1299)

**NOTE**: the rows and cols of raster data are opposite of the rows and cols of mathimatical matrix. 

For example, here RasterXSize is 1299, means there are columns in the matrix. Thus, taking a pixel from data array using offsets should be data[yoffset, xoffset].

#### Get Projection Information

In [44]:
prj = ds.GetProjection() # get the projection info in WKT
prj

'PROJCS["unnamed",GEOGCS["unknown",DATUM["unknown",SPHEROID["unnamed",6371228,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",90],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'

#### Unprojecting projection coordinate system to geographic coordinate system
There might be multiple ways to achieve the unprojection.
#### 1. osgeo
**WGS-84 geographic spatial reference system(EPSG 4326)**

In [48]:
from osgeo import osr

# get the original GeoTIFFSpatialReference
tifSpatialRef = osr.SpatialReference()
tifSpatialRef.ImportFromWkt(prj)

0

In [60]:
# Define a new spatial reference, which is the lat/lon geographic spatial reference
geoSpatialRef = osr.SpatialReference()
geoSpatialRef.ImportFromEPSG(4326)

# define the coordinate transformation
transform = osr.CoordinateTransformation(tifSpatialRef, geoSpatialRef)

transform.TransformPoint(xcoordinates[0], ycoordinates[0])

(-94.99774138723588, 39.787958548172064, 0.0)

But the ```TransformPoint``` has to be done point by point. So it's not convinient. 

#### 2. PyProj

In [71]:
import pyproj

proj4 = tifSpatialRef.ExportToProj4()
inproj = pyproj.Proj(proj4)

xv, yv = np.meshgrid(xcoordinates, ycoordinates)
lons, lats = inproj(xv, yv, inverse=True)
# the inverse tag is for indicating the direction of projection
# lat/lon to other coordinate systems: inverse=False
# other to lat/lon: inverse=True

#### Other packages that might be useful
* [rasterio](http://xarray.pydata.org/en/stable/auto_gallery/plot_rasterio.html#sphx-glr-auto-gallery-plot-rasterio-py)
* [xarray](http://xarray.pydata.org/en/stable/generated/xarray.open_rasterio.html)
* [affine]
* [glob]

In [73]:
yv

array([[  471000.,   471000.,   471000., ...,   471000.,   471000.,
          471000.],
       [  470000.,   470000.,   470000., ...,   470000.,   470000.,
          470000.],
       [  469000.,   469000.,   469000., ...,   469000.,   469000.,
          469000.],
       ...,
       [-1482000., -1482000., -1482000., ..., -1482000., -1482000.,
        -1482000.],
       [-1483000., -1483000., -1483000., ..., -1483000., -1483000.,
        -1483000.],
       [-1484000., -1484000., -1484000., ..., -1484000., -1484000.,
        -1484000.]])

In [74]:
np.rot90(yv)

array([[  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.],
       [  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.],
       [  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.],
       ...,
       [  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.],
       [  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.],
       [  471000.,   470000.,   469000., ..., -1482000., -1483000.,
        -1484000.]])

In [77]:
len(data)

1956

In [78]:
len(np.rot90(data))

1299